In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#pip install pandas_ta

In [3]:
import pandas_ta as ta

In [4]:
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [21]:
bist_daily = pd.read_csv("datasets/bist_100_daily.csv")
# Extract the number of rows and columns by using the shape of the data.
numRows,numColumns = bist_daily.shape
# Extract the time interval.
last_date, first_date = bist_daily.iloc[0].Date, bist_daily.iloc[-1].Date
# Check the availability of the data.
na_cols = bist_daily.columns[bist_daily.isna().any()].tolist()

# Print the information.
print(f"There are {numRows} rows and {numColumns} columns in the initial dataset.")
print(f"The data represents the time frame between the dates '{last_date}' and '{first_date}'.")
if not na_cols:
    print("There are no NA rows.")
else:
    print(f"Columns in the dataset which include NA rows: {na_cols}.")
# Convert columns to numeric values
column_names = ["Price", "Open", "High", "Low"]
for column in column_names:
    bist_daily[column] = bist_daily[column].str.replace(',', '')
    bist_daily[column] = pd.to_numeric(bist_daily[column])
# CONVERT TO DATETIME FORMAT AND SORT DATA BY DATE
bist_daily.Date = pd.to_datetime(bist_daily.Date)
bist_daily.sort_values(by="Date", ignore_index=True,inplace=True)
bist_daily.set_index(pd.DatetimeIndex(bist_daily["Date"]), inplace=True)
bist_daily.rename(columns={"Price": "close"},inplace=True)
# Calculate Returns and append to the df DataFrame
# CUMLOGRET_1 and CUMPCTRET_1 are added (NaN values exists)
bist_daily.ta.log_return(cumulative=True, append=True)
bist_daily.ta.percent_return(cumulative=True, append=True)
# Returns a list of indicators and utility functions (to check in future)
ind_list = bist_daily.ta.indicators(as_list=True)
# RSI_14, MACD_12_26_9, MACDh_12_26_9 and MACDs_12_26_9 are added (NaN values exists)
bist_daily.ta.rsi(append=True)
bist_daily.ta.macd(append=True)
# SMA values are added (use ta in the future)
sma_values = [5, 10, 15] 
for i in sma_values:
    bist_daily['SMA'+str(i)] = bist_daily['close'].rolling(window=i).mean()
# Remove all NaN value rows
bist_daily.dropna(inplace=True)
bist_daily

There are 5000 rows and 7 columns in the initial dataset.
The data represents the time frame between the dates 'Dec 11, 2019' and 'Jan 04, 2000'.
There are no NA rows.


,date,close,open,high,low,Vol.,Change %,CUMLOGRET_1,CUMPCTRET_1,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,SMA5,SMA10,SMA15
date,,,,,,,,,,,,,,,,
2000-02-21,2000-02-21,145.69,153.64,153.64,142.79,25.75M,-5.17%,-0.183990,-0.160515,34.352566,-7.230371,-0.392050,-6.838321,153.936,155.116,157.896000
2000-02-22,2000-02-22,139.94,145.69,147.02,137.58,29.58M,-3.95%,-0.224258,-0.199983,31.136786,-8.027940,-0.951695,-7.076245,150.206,153.060,156.330667
2000-02-23,2000-02-23,134.47,139.94,141.26,131.71,26.95M,-3.91%,-0.264130,-0.239071,28.411996,-8.997682,-1.537150,-7.460532,145.432,151.074,154.602000
2000-02-24,2000-02-24,146.52,134.47,146.52,134.14,43.02M,8.96%,-0.178310,-0.149460,40.719161,-8.693661,-0.986503,-7.707158,144.052,149.896,153.638667
2000-02-25,2000-02-25,156.18,146.52,156.32,146.52,56.16M,6.59%,-0.114462,-0.083530,48.380526,-7.585796,0.097090,-7.682886,144.560,149.916,152.803333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-05,2019-12-05,1086.59,1082.60,1090.32,1081.51,2.24B,0.89%,1.825328,2.894902,66.153619,16.788401,0.360019,16.428382,1076.116,1068.981,1068.217333
2019-12-06,2019-12-06,1088.69,1090.09,1093.15,1083.52,2.16B,0.19%,1.827259,2.896834,66.774984,17.187282,0.607120,16.580162,1080.046,1071.262,1070.543333
2019-12-09,2019-12-09,1087.86,1088.96,1091.25,1081.39,2.32B,-0.08%,1.826496,2.896072,66.257244,17.237719,0.526046,16.711673,1081.610,1074.666,1071.836667


In [22]:
# edit Vol. column

from operator import itemgetter

vols = bist_daily['Vol.'].to_list()
#[vol[-1] for vol in vols]
#list(filter(lambda vol: "-" in vol,enumerate(vols)))

indexToRemove = bist_daily.iloc[list(map(itemgetter(0),filter(lambda vol: "-" in vol,enumerate(vols))))].index
bist_daily.drop(indexToRemove,inplace=True)
bist_daily['Vol.'] = bist_daily['Vol.'].apply(
    lambda x: float(x[:-1])*(10**6) if x[-1]=="M" else (float(x[:-1])*(10**9) if x[-1]=="B" else "ERROR")).astype("int")
bist_daily

,date,close,open,high,low,Vol.,Change %,CUMLOGRET_1,CUMPCTRET_1,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,SMA5,SMA10,SMA15
date,,,,,,,,,,,,,,,,
2000-02-21,2000-02-21,145.69,153.64,153.64,142.79,25750000,-5.17%,-0.183990,-0.160515,34.352566,-7.230371,-0.392050,-6.838321,153.936,155.116,157.896000
2000-02-22,2000-02-22,139.94,145.69,147.02,137.58,29580000,-3.95%,-0.224258,-0.199983,31.136786,-8.027940,-0.951695,-7.076245,150.206,153.060,156.330667
2000-02-23,2000-02-23,134.47,139.94,141.26,131.71,26950000,-3.91%,-0.264130,-0.239071,28.411996,-8.997682,-1.537150,-7.460532,145.432,151.074,154.602000
2000-02-24,2000-02-24,146.52,134.47,146.52,134.14,43020000,8.96%,-0.178310,-0.149460,40.719161,-8.693661,-0.986503,-7.707158,144.052,149.896,153.638667
2000-02-25,2000-02-25,156.18,146.52,156.32,146.52,56160000,6.59%,-0.114462,-0.083530,48.380526,-7.585796,0.097090,-7.682886,144.560,149.916,152.803333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-05,2019-12-05,1086.59,1082.60,1090.32,1081.51,2240000000,0.89%,1.825328,2.894902,66.153619,16.788401,0.360019,16.428382,1076.116,1068.981,1068.217333
2019-12-06,2019-12-06,1088.69,1090.09,1093.15,1083.52,2160000000,0.19%,1.827259,2.896834,66.774984,17.187282,0.607120,16.580162,1080.046,1071.262,1070.543333
2019-12-09,2019-12-09,1087.86,1088.96,1091.25,1081.39,2320000000,-0.08%,1.826496,2.896072,66.257244,17.237719,0.526046,16.711673,1081.610,1074.666,1071.836667


In [23]:
# seeding an arbitrary number to get same results in multiple runs
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)
print("Seed:", manualSeed)

Seed: 999


In [24]:
# getting number of GPUs from cuda
ngpu = torch.cuda.device_count()
print("Count of available GPUs:", ngpu)

Count of available GPUs: 1


In [25]:
# printing the name of available GPUs
for i in range(ngpu):
    print("GPU {}: {}".format(i+1, torch.cuda.get_device_name(i)))

GPU 1: GeForce MX150


In [26]:
# batch size for the training
batch_size = 64

# optimizer rates
optimizer_betas = (0.9, 0.999)
learning_rate = 1e-3

# number of epochs
num_epochs = 100000

# decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [27]:
class TimeseriesDataset(Dataset):
    def __init__(self, data_frame, sequence_length=2):
        self.data = torch.tensor(data_frame.values)
        self.sequence_length = sequence_length

    def __len__(self):
        return self.data.shape[0] - self.sequence_length + 1

    def __getitem__(self, index):
        return self.data[index: index + self.sequence_length].float()
    
    # Non-overlapping series
    # def __getitem__(self, index):
    #     return self.data[index * self.sequence_length: (index+1) * self.sequence_length]

In [28]:
# create pytorch dataset from the pandas DataFrame

# TODO: Convert change(%) and Volume columns to numeric values
columns_used_in_training = ["close", "open", "high", "low", "CUMLOGRET_1", "RSI_14", "MACD_12_26_9", "SMA5"]
# input dimension of the generator
data_dimension = len(columns_used_in_training)
# sequence length of input data
sequence_length = 30
train_dataset = TimeseriesDataset(bist_daily[columns_used_in_training], sequence_length)
# create the dataloader
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
real_data_sample = next(iter(dataloader))
print("Real data sample shape:", real_data_sample.shape)

Real data sample shape: torch.Size([64, 30, 8])


In [29]:
class Generator(nn.Module):
    def __init__(self, hidden_size):
        super(Generator, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=data_dimension, hidden_size=hidden_size, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_size, data_dimension)
        self.drop = nn.Dropout(0.2)

    def forward(self, input_sequences):
        input_sequences = self.drop(input_sequences)
        lstm_output, hidden_cell = self.lstm(input_sequences)
        res = self.linear(hidden_cell[0][-1])
        res = res.view(res.shape[0], 1, -1)
        return res

In [30]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(sequence_length*data_dimension, 72),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(72, 100),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(100, 10),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(10, 1),
            nn.Sigmoid(),
        )

    def forward(self, input_sequences):
        input_sequences_flattened = input_sequences.view(input_sequences.shape[0], -1)
        res = self.model(input_sequences_flattened)
        return res

In [31]:
# TODO: weight initialization of models

In [32]:
generator = Generator(hidden_size=data_dimension*2).to(device)
discriminator = Discriminator().to(device)
print("Generator and discriminator are initialized")

Generator and discriminator are initialized


In [33]:
criterion = nn.BCELoss()
optimizer_generator = optim.Adam(generator.parameters(), lr=learning_rate, betas=optimizer_betas)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=optimizer_betas)

real_label = 1.
fake_label = 0.

In [34]:
print("Training is started")
for epoch in range(num_epochs):
    for i, sequence_batch in enumerate(dataloader):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Training with real batch
            discriminator.zero_grad()
            # Format batch
            real_sequence = sequence_batch.to(device)
            batch_size = real_sequence.size(0)
            real_labels = torch.full((batch_size,), real_label, dtype=torch.float, device=device)
            # Forward pass real batch through D
            discriminator_output_real = discriminator(real_sequence).view(-1)
            # Calculate loss on all-real batch
            discriminator_error_real = criterion(discriminator_output_real, real_labels)
            # Calculate gradients for D in backward pass
            discriminator_error_real.backward()

            ## Training with fake batch
            # Assign first t values
            generator_input_sequence = sequence_batch[:,:-1].to(device)
            #  Generate (t+1)th value from first t values
            generated_values = generator(generator_input_sequence)
            fake_labels = torch.full((batch_size,), fake_label, dtype=torch.float, device=device)
            # Concat first t real values and generated (t+1)th values
            generator_result_concat = torch.cat((generator_input_sequence, generated_values.detach()), 1)
            # Classify all fake batch with D
            discriminator_output_fake = discriminator(generator_result_concat).view(-1)
            # Calculate D's loss on the all-fake batch
            discriminator_error_fake = criterion(discriminator_output_fake, fake_labels)
            # Calculate the gradients for this batch
            discriminator_error_fake.backward()
            # Add the gradients from the all-real and all-fake batches
            discriminator_error = discriminator_error_real + discriminator_error_fake
            # Update D
            optimizer_discriminator.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            generator.zero_grad()
            real_labels = torch.full((batch_size,), real_label, dtype=torch.float, device=device)
            # Since we just updated D, perform another forward pass of all-fake batch through D
            generator_result_concat_grad = torch.cat((generator_input_sequence, generated_values), 1)
            discriminator_output_fake = discriminator(generator_result_concat_grad).view(-1)
            # Calculate G's loss based on this output
            generator_error = criterion(discriminator_output_fake, real_labels)
            # Calculate gradients for G
            generator_error.backward()
            # Update G
            optimizer_generator.step()
    if (epoch+1) % 50 == 0 or epoch+1 == 1:
        print('\n[{}/{}]\tDiscriminator Loss: {:.4f}\tGenerator Loss: {:.4f}'
                  .format(epoch+1, num_epochs, discriminator_error.item(), generator_error.item()))
        for col_name, real, generated in zip(columns_used_in_training, sequence_batch[0][-1], generated_values[0][0]):
            print(f"{col_name} | Real:{real:.4f} / Generated:{generated:.4f}")
       

Training is started

[1/100000]	Discriminator Loss: 0.0168	Generator Loss: 25.0191
close | Real:611.8200 / Generated:1.6335
open | Real:613.2700 / Generated:0.8719
high | Real:620.0300 / Generated:1.3284
low | Real:610.3700 / Generated:0.8923
CUMLOGRET_1 | Real:1.2510 / Generated:0.9351
RSI_14 | Real:42.5171 / Generated:0.6384
MACD_12_26_9 | Real:-5.8408 / Generated:0.8388
SMA5 | Real:608.6060 / Generated:1.8327

[50/100000]	Discriminator Loss: 0.0000	Generator Loss: 65.3816
close | Real:543.2000 / Generated:75.1471
open | Real:541.1600 / Generated:76.5208
high | Real:545.7500 / Generated:76.4702
low | Real:538.8300 / Generated:74.7241
CUMLOGRET_1 | Real:1.1320 / Generated:4.4167
RSI_14 | Real:51.4258 / Generated:53.3587
MACD_12_26_9 | Real:-4.9625 / Generated:-11.5432
SMA5 | Real:532.7400 / Generated:77.0572

[100/100000]	Discriminator Loss: 0.1708	Generator Loss: 65.0238
close | Real:232.9000 / Generated:115.9710
open | Real:234.1800 / Generated:116.5135
high | Real:235.5200 / Genera


[1050/100000]	Discriminator Loss: 0.2764	Generator Loss: 61.8901
close | Real:885.3300 / Generated:428.7156
open | Real:894.8000 / Generated:435.5400
high | Real:898.5500 / Generated:438.6423
low | Real:883.1600 / Generated:425.7332
CUMLOGRET_1 | Real:1.6205 / Generated:2.2621
RSI_14 | Real:26.2398 / Generated:39.8018
MACD_12_26_9 | Real:-21.1445 / Generated:0.2137
SMA5 | Real:903.9100 / Generated:436.9734

[1100/100000]	Discriminator Loss: 0.6303	Generator Loss: 43.1280
close | Real:204.8500 / Generated:202.1545
open | Real:203.2200 / Generated:200.1810
high | Real:205.5000 / Generated:204.3104
low | Real:203.2200 / Generated:198.1946
CUMLOGRET_1 | Real:0.1568 / Generated:-1.1920
RSI_14 | Real:61.1540 / Generated:61.7239
MACD_12_26_9 | Real:4.0385 / Generated:4.1237
SMA5 | Real:203.8320 / Generated:202.6025

[1150/100000]	Discriminator Loss: 0.7507	Generator Loss: 47.7015
close | Real:955.8800 / Generated:442.6130
open | Real:955.6900 / Generated:448.7006
high | Real:958.2500 / Gener


[2050/100000]	Discriminator Loss: 0.0636	Generator Loss: 56.7532
close | Real:948.8700 / Generated:521.5678
open | Real:947.3600 / Generated:559.0026
high | Real:953.7500 / Generated:554.0084
low | Real:945.3800 / Generated:534.3890
CUMLOGRET_1 | Real:1.6898 / Generated:-0.1104
RSI_14 | Real:55.2958 / Generated:24.6064
MACD_12_26_9 | Real:1.1565 / Generated:-1.7093
SMA5 | Real:941.2820 / Generated:561.4235

[2100/100000]	Discriminator Loss: 0.5279	Generator Loss: 36.0779
close | Real:105.9500 / Generated:111.3761
open | Real:105.2300 / Generated:102.5538
high | Real:106.6000 / Generated:107.7933
low | Real:104.8000 / Generated:104.9408
CUMLOGRET_1 | Real:-0.5025 / Generated:-0.9489
RSI_14 | Real:59.6383 / Generated:60.7097
MACD_12_26_9 | Real:2.5833 / Generated:3.0184
SMA5 | Real:106.2760 / Generated:102.0332

[2150/100000]	Discriminator Loss: 0.0743	Generator Loss: 55.7138
close | Real:765.3100 / Generated:523.8521
open | Real:764.8400 / Generated:560.9478
high | Real:770.4400 / Gene


[3050/100000]	Discriminator Loss: 0.3812	Generator Loss: 51.5193
close | Real:647.3000 / Generated:593.8888
open | Real:641.3100 / Generated:615.3970
high | Real:647.3000 / Generated:616.1976
low | Real:639.9500 / Generated:600.7843
CUMLOGRET_1 | Real:1.3073 / Generated:1.5394
RSI_14 | Real:45.9227 / Generated:39.7068
MACD_12_26_9 | Real:-4.9627 / Generated:-4.7392
SMA5 | Real:649.7640 / Generated:621.7347

[3100/100000]	Discriminator Loss: 0.1782	Generator Loss: 64.4786
close | Real:245.6000 / Generated:261.2206
open | Real:241.3800 / Generated:244.1698
high | Real:247.6500 / Generated:255.1410
low | Real:241.3800 / Generated:250.8779
CUMLOGRET_1 | Real:0.3382 / Generated:-0.2350
RSI_14 | Real:48.6560 / Generated:56.6876
MACD_12_26_9 | Real:-3.7692 / Generated:0.0054
SMA5 | Real:240.1260 / Generated:245.6909

[3150/100000]	Discriminator Loss: 0.2498	Generator Loss: 63.1312
close | Real:346.0100 / Generated:353.9337
open | Real:348.0800 / Generated:328.1557
high | Real:348.4100 / Gene


[4050/100000]	Discriminator Loss: 0.2331	Generator Loss: 39.6407
close | Real:268.1100 / Generated:279.2838
open | Real:265.9800 / Generated:268.6082
high | Real:268.2600 / Generated:276.6305
low | Real:265.4900 / Generated:271.3177
CUMLOGRET_1 | Real:0.4259 / Generated:0.3250
RSI_14 | Real:62.9210 / Generated:59.5854
MACD_12_26_9 | Real:4.8508 / Generated:4.7527
SMA5 | Real:268.4860 / Generated:269.7894

[4100/100000]	Discriminator Loss: 0.0612	Generator Loss: 44.1319
close | Real:754.0600 / Generated:654.8165
open | Real:753.0700 / Generated:666.9028
high | Real:756.1500 / Generated:670.7235
low | Real:744.7400 / Generated:654.3729
CUMLOGRET_1 | Real:1.4600 / Generated:1.7435
RSI_14 | Real:46.2507 / Generated:46.1035
MACD_12_26_9 | Real:-9.3500 / Generated:-2.8452
SMA5 | Real:747.1780 / Generated:681.0638

[4150/100000]	Discriminator Loss: 0.2557	Generator Loss: 39.9975
close | Real:551.4800 / Generated:537.0791
open | Real:556.1600 / Generated:522.9736
high | Real:556.5300 / Genera

...


[55050/100000]	Discriminator Loss: 0.0355	Generator Loss: 90.0734
close | Real:93.5100 / Generated:88.8173
open | Real:90.6000 / Generated:88.4504
high | Real:94.5200 / Generated:91.0815
low | Real:87.9900 / Generated:87.8889
CUMLOGRET_1 | Real:-0.6274 / Generated:-0.4111
RSI_14 | Real:45.2934 / Generated:36.1363
MACD_12_26_9 | Real:-3.4669 / Generated:-4.4705
SMA5 | Real:92.2380 / Generated:89.8277

[55100/100000]	Discriminator Loss: 0.0083	Generator Loss: 87.8826
close | Real:360.5100 / Generated:255.3978
open | Real:362.4600 / Generated:251.7598
high | Real:365.4100 / Generated:257.5905
low | Real:356.1600 / Generated:251.1013
CUMLOGRET_1 | Real:0.7220 / Generated:0.5533
RSI_14 | Real:44.8002 / Generated:45.2157
MACD_12_26_9 | Real:-10.4139 / Generated:-4.9615
SMA5 | Real:359.2020 / Generated:250.5237

[55150/100000]	Discriminator Loss: 0.0043	Generator Loss: 87.1322
close | Real:775.3500 / Generated:273.6366
open | Real:776.0700 / Generated:273.3337
high | Real:777.8100 / Generate


[56050/100000]	Discriminator Loss: 0.0121	Generator Loss: 83.7239
close | Real:770.3500 / Generated:186.3796
open | Real:779.1200 / Generated:182.4451
high | Real:780.4300 / Generated:189.8424
low | Real:770.2200 / Generated:175.9919
CUMLOGRET_1 | Real:1.4814 / Generated:0.1550
RSI_14 | Real:37.9985 / Generated:60.6978
MACD_12_26_9 | Real:-10.9295 / Generated:-2.9813
SMA5 | Real:778.8760 / Generated:185.7840

[56100/100000]	Discriminator Loss: 0.0728	Generator Loss: 85.3765
close | Real:337.6200 / Generated:252.8061
open | Real:346.0100 / Generated:250.5703
high | Real:349.0300 / Generated:252.9789
low | Real:334.2200 / Generated:245.5683
CUMLOGRET_1 | Real:0.6564 / Generated:0.5405
RSI_14 | Real:35.1702 / Generated:45.1348
MACD_12_26_9 | Real:-18.0343 / Generated:-4.6801
SMA5 | Real:341.6920 / Generated:242.8790

[56150/100000]	Discriminator Loss: 0.0110	Generator Loss: 88.9611
close | Real:625.5300 / Generated:238.0531
open | Real:620.2100 / Generated:242.1393
high | Real:629.5000 /


[57050/100000]	Discriminator Loss: 0.2087	Generator Loss: 58.9381
close | Real:90.0800 / Generated:94.9420
open | Real:93.9900 / Generated:94.7187
high | Real:93.9900 / Generated:96.4257
low | Real:88.7100 / Generated:91.6310
CUMLOGRET_1 | Real:-0.6648 / Generated:-0.7043
RSI_14 | Real:36.5509 / Generated:43.4633
MACD_12_26_9 | Real:-1.8005 / Generated:-1.6310
SMA5 | Real:93.8860 / Generated:95.2362

[57100/100000]	Discriminator Loss: 0.0737	Generator Loss: 68.8375
close | Real:530.6200 / Generated:233.0686
open | Real:512.9300 / Generated:231.4418
high | Real:530.7500 / Generated:236.7836
low | Real:499.3100 / Generated:232.7259
CUMLOGRET_1 | Real:1.1086 / Generated:0.5695
RSI_14 | Real:36.4179 / Generated:37.0267
MACD_12_26_9 | Real:-24.9143 / Generated:-5.8418
SMA5 | Real:533.6160 / Generated:231.9610

[57150/100000]	Discriminator Loss: 0.2522	Generator Loss: 86.5040
close | Real:196.1100 / Generated:191.4490
open | Real:192.9400 / Generated:189.4303
high | Real:196.1900 / Generate


[58050/100000]	Discriminator Loss: 0.1703	Generator Loss: 72.1664
close | Real:354.0700 / Generated:293.5610
open | Real:358.0200 / Generated:292.7358
high | Real:360.2700 / Generated:298.9424
low | Real:352.7700 / Generated:288.8878
CUMLOGRET_1 | Real:0.7040 / Generated:0.7755
RSI_14 | Real:60.3399 / Generated:66.7156
MACD_12_26_9 | Real:6.7356 / Generated:6.6580
SMA5 | Real:352.1500 / Generated:291.8302

[58100/100000]	Discriminator Loss: 0.0001	Generator Loss: 100.0000
close | Real:818.3500 / Generated:264.9938
open | Real:823.4800 / Generated:264.6811
high | Real:827.1800 / Generated:270.5897
low | Real:815.4600 / Generated:260.2464
CUMLOGRET_1 | Real:1.5418 / Generated:0.2789
RSI_14 | Real:61.2935 / Generated:59.6097
MACD_12_26_9 | Real:3.5417 / Generated:0.9918
SMA5 | Real:811.0800 / Generated:264.3176

[58150/100000]	Discriminator Loss: 0.0012	Generator Loss: 92.2700
close | Real:605.5700 / Generated:276.1567
open | Real:598.0700 / Generated:270.3479
high | Real:607.3000 / Gene

KeyboardInterrupt: 